In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("PySparkOnYarn") \
    .master('yarn') \
    .config("spark.submit.deployMode", "client") \
    .config("spark.driver.cores", 1) \
    .config("spark.driver.memory", "1G") \
    .config("spark.executor.cores", 1) \
    .config("spark.executor.memory", "1G") \
    .config("spark.executor.num", 2) \
    .config("spark.jars","hdfs://node01:9000/jar/mysql-connector-java-8.0.16.jar") \
    .enableHiveSupport() \
    .getOrCreate()

In [29]:

tb_df = spark.read \
    .format("jdbc") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://node01:3306/txkt") \
    .option("user", "root") \
    .option("password", "@WSX3edc") \
    .option("spark.sql.parquet.int96AsTimestamp",False) \
    .option("dbtable", 'tencent_study_video_spider_2022_03_26') \
    .load()
tb_df.createOrReplaceTempView("mysql_tencent_study_video_spider")

In [ ]:
spark.catalog.cacheTable('tencent_study_video_spider_2022_03_23')

In [5]:
spark.sql(
    '''
    CREATE TABLE IF NOT EXISTS txkt.ods_tencent_study_video
      (
        `video_type1`          STRING COMMENT '视频大类',
        `video_type2`          STRING COMMENT '视频中类',
        `video_type3`          STRING COMMENT '视频小类',
        `video_title`          STRING COMMENT '视频标题',
        `video_url`            STRING COMMENT '视频url',
        `video_sections_num`   STRING COMMENT '视频节数',
        `video_index_page`     INT    COMMENT '视频所在页数',
        `organ_name`           STRING COMMENT '机构名称',
        `price`                STRING COMMENT '收费信息(单位：分)',
        `person_num`           STRING COMMENT '购买/报名人数',
        `recently_study_num`   STRING COMMENT '最近在学',
        `sign_up_num`          STRING COMMENT '累计报名',
        `video_praise_degree`  STRING COMMENT '好评度',
        `organ_praise_degree`  STRING COMMENT '机构好评度',
        `organ_video_num`      STRING COMMENT '机构课程数',
        `organ_all_person_num` STRING COMMENT '机构总学习人次',
        `grab_time`            STRING COMMENT '数据采集时间',
        `etl_time`             STRING COMMENT '数据加工时间'
      ) 
      PARTITIONED BY (`dt` STRING) 
      STORED AS PARQUET
    '''
)


DataFrame[]

In [30]:
spark.sql(
    'INSERT OVERWRITE TABLE txkt.ods_tencent_study_video PARTITION (dt)' + 
    '''
         SELECT video_type1,         
                video_type2,         
                video_type3,         
                video_title,         
                video_url,           
                video_sections_num,  
                video_index_page,    
                organ_name,          
                price,               
                person_num,          
                recently_study_num,  
                sign_up_num,         
                video_praise_degree, 
                organ_praise_degree, 
                organ_video_num,     
                organ_all_person_num,
                grab_time,           
                FROM_UNIXTIME(CAST(NOW() AS BIGINT), 'yyyy-MM-dd HH:mm:ss') AS elt_time,
                DATE_FORMAT(TO_TIMESTAMP(grab_time, 'yyyy-MM-dd HH:mm:ss'), 'yyyy-MM-dd') AS dt
           FROM mysql_tencent_study_video_spider
    '''
)

DataFrame[]

In [32]:
spark.sql("SELECT dt, COUNT(*) FROM txkt.ods_tencent_study_video GROUP BY dt ORDER BY dt;").show()

+----------+--------+
|        dt|count(1)|
+----------+--------+
|2022-03-13|   77566|
|2022-03-23|   77995|
|2022-03-24|   78161|
|2022-03-25|   77030|
|2022-03-26|   78488|
+----------+--------+



In [ ]:
spark.stop()